In [1]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.graph_objects import Layout
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Image

# Use Electricity data when reducing your carbon footprint!

<img src="electricitymap-logo.svg" width=600 height=600 />


## Why are there two electricity zones in Denmark (East Denmark and West Denmark)?

In Denmark there are two non-synchronous electricity zones, connected through the great belt power link (<i>Wang, Zong et al. 2017, p23</i>). West Denmark (DK1)  is part of the European continental electricity system and East Denmark (DK2)is part of the Nordic electricity system. 

Why?:

- Situated between predominantly hydro Nordic electricity system and thermal central European electricity system 
    $\rightarrow$  Denmark plays important role in transporting energy between these systems
- Situated between low electricity prices in the North and high electricity prices in central Europe $\rightarrow$ Optimal space for trading 
- Connections to these electricity systems leads to robust capacity adequacy


<i>(Danish Energy Agency: Security of Electricity Supply in Denmark. Working group report on methodology, concepts and calculations concerning security of electricity supply, 2015)</i>


Overview of the connections

<img src="energinet.png" width=600 height=600 />

## Access data from database

In [2]:
conn = sqlite3.connect('energy.db')
cursor = conn.cursor()

In [3]:
carbon = pd.read_sql("SELECT * FROM carbon", conn)
prod = pd.read_sql("SELECT * FROM production", conn)

In [4]:
cursor.close()
conn.close()

## Important measures when wanting to reduce your carbon footprint

- <B>Carbon Itensity</B> : number of grams of carbon dioxide (CO2) it takes to make one unit of electricity a kilowatt per hour (kW/hour)
- <B>Carbon Footprint </B> : Carbon footprint is measured as how many tons of carbon dioxide are emitted per year. Often we talk about our personal Carbon Footprint. According to the latest numbers of worldometers a Danish person,  on average, has a carbon footprint of 6.65 Tons per year.  
- <B>Capacity </B> :  the maximum output of electricity that a generator can produce under ideal conditions. Measured in Watt or (megawatts, kilowatts)
- <B>Generation</B> : The amount of electricity that is produced over a specific period of time. Measured in watt per hour (or Kwh, Mwh)

## Carbon intensity of Energy Production types in East Denmark

In [5]:
# General layout stuff for a visuaizations
layout = {
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
'font_family': 'Courier New'
}

In [88]:
#only values for production types that exist in East Denmark
carbon = carbon[carbon['energy_type'].isin(prod.columns)]

fig = px.bar(carbon, x='energy_type', y='value', color = 'value',
            labels={'value':'Carbon Intensity', 'energy_type': 'Production type'},
            color_continuous_scale='Geyser').update_xaxes(categoryorder="total ascending")

fig.add_shape(type="rect",
    x0=-0.4, y0=0, x1=2.5, y1=300,
    line=dict(color="Green", width=3),
)

fig.update_layout(layout)

fig.update_layout(
    title="Mean Carbon Intensity of the different energy production types in Denmark",
    xaxis_title="Energy Production Type",
    yaxis_title="Carbon Intensity - gCO2 per KWh")

fig.show()


## Total energy Production per type

In [89]:
#little dataframe with total production pr energy type as well as percentage of total production 
total = pd.DataFrame(prod.loc[:,'gas':'wind'].sum()).reset_index()
total.columns = ['Energy_type', 'Yearly_production']
total['Percentage'] = round((total['Yearly_production'] / sum(total['Yearly_production']))*100, 2)
total = total.merge(carbon, left_on='Energy_type', right_on='energy_type')

In [91]:
fig = px.bar(total, x='Percentage', y='Energy_type', color='value',color_continuous_scale='Geyser',
            labels={'Percentage':'Percentage', 'Energy_type': 'Production type',
                   'value': 'Carbon Intensity'}).update_yaxes(categoryorder="total ascending", title="Energy Type")

fig.update_layout(layout)

fig.update_layout(
    title="Percentage of production types of total energy production in East Denmark - 2020")

fig.show()

## When is the carbon footprint the lowest during the day

In [146]:
# electricity production by hour 
pr = prod.groupby(['time']).mean()

# Calculate gram/kwh to ton/kwh
carbon['ton/kwh'] = carbon['value'] / (1000*1000)

# new dataframe and calculate MW to KW
footprint = pr
footprint = footprint*1000


#Add carbon intensity to electricity production (probably not the smartest way)
footprint['gas'] = footprint['gas'] * carbon['ton/kwh'].iloc[2]
footprint['coal'] = footprint['coal'] * carbon['ton/kwh'].iloc[1]

footprint['oil'] = footprint['oil'] * carbon['ton/kwh'].iloc[3]
footprint['solar'] = footprint['solar'] * carbon['ton/kwh'].iloc[4]
footprint['biomass'] = footprint['biomass'] * carbon['ton/kwh'].iloc[0]
footprint['wind'] = footprint['wind'] * carbon['ton/kwh'].iloc[5]

#total footprint 
footprint["total"] = footprint.sum(axis=1)

#difference throughout the day 
max_foot = round(footprint['total'].max(),2)
min_foot = round(footprint['total'].min(),2)
difference = round(max_foot - min_foot,2)

In [138]:
fig = px.bar(round(footprint['total'],2), labels={'variable':'Energy Type', 'time': 'Time of day',
                   'value': 'Carbon footprint'}, color='value',color_continuous_scale='Geyser')

fig.update_layout(layout)

fig.update_layout(
    title="Differences in Carbon footprint throughout the day",
    xaxis_title="Time of day",
    yaxis_title="Carbon Emission - Tons",
    showlegend=False)


fig.show()

Maximum Carbon footprint during the day is 310.52 tons CO2. at 19:00

Minimum Carbon footprint during the day is 249.71 tons CO2. at 02:00

The difference is 60.81 tons. 

In [172]:


pr["total"] = pr.sum(axis=1)
foot_pr = ((pr.iloc[:,:-1].div(pr['total'], axis=0))*100).T


## Minimum Carbon footprint - Energy composition

In [171]:
fig = px.bar(round(foot_pr['02:00'],2), color_discrete_sequence =['rgb(202, 86, 44)']*len(foot_pr))

fig.update_layout(layout)

fig.update_layout(
    title="Compostion of energy types at 02.00 - Minimum Carbon Footprint",
    xaxis_title="Energy Type",
    yaxis_title="Percentage",
    showlegend=False)


fig.show()

## Maximum Carbon footprint - Energy composition 

In [170]:
fig = px.bar(round(foot_pr['19:00'],2), color_discrete_sequence =['rgb(202, 86, 44)']*len(foot_pr))

fig.update_layout(layout)

fig.update_layout(
    title="Compostion of energy types at 19.00 - Maximum Carbon Footprint",
    xaxis_title="Energy Type",
    yaxis_title="Percentage",
    showlegend=False)


fig.show()

In [ ]:
['rgb(0, 128, 128)', 'rgb(112, 164, 148)', 'rgb(180, 200, 168)', 'rgb(246, 237, 189)', 'rgb(237, 187, 138)', 'rgb(222, 138, 90)', 'rgb(202, 86, 44)']





## Best time of the day  - Extra stuff 

In [84]:
def visualize_day(energy_type):
    
    fig = px.bar(pr, x=pr.index, y=energy_type)


    fig.update_layout(layout)

    return fig.show()

widgets.interact(visualize_day, energy_type =pr.columns);

interactive(children=(Dropdown(description='energy_type', options=('gas', 'coal', 'oil', 'solar', 'biomass', '…